In [1]:
import redis
import msgpack
import msgpack_numpy as mnp
pool = redis.ConnectionPool(host='34.69.39.105', port=6379, db=0)
r = redis.Redis(connection_pool=pool)

In [43]:
import numpy as np
from bokeh.models import Circle, ColumnDataSource,Plot,ColorBar, HoverTool, CustomJS, AjaxDataSource
from bokeh.plotting import figure, save
from bokeh.transform import linear_cmap
from bokeh.palettes import YlOrBr
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.io import show, output_notebook, output_file
from bokeh.events import Event, LODEnd
import geopandas as gpd
import requests

In [3]:
def merc(lon, lat):
    ''' Get mercator projection co-ordinates from latitude and logitude'''
    x=[]
    y=[]
    r_major = 6378137.000
    for i in range(len(lon)):
        x.append(r_major * np.radians(lon[i])) 
        scale = x[i]/lon[i]
        y.append(180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
         lat[i]* (np.pi/180.0)/2.0)) * scale)
    return x, y

In [4]:
def lonlat(x, y):
    lon = []
    lat = []
    r_major = 6378137.000
    for i in range(len(x)):
        lon.append(np.degrees(x[i]/r_major))
        scale = lon[i]/x[i]
        lat.append(2.0*(np.arctan(np.exp(np.pi*y[i]/180.0*scale))-np.pi/4.0)*180.0/np.pi)
    return lon, lat

In [5]:
# Get data from redis server
predmeans = mnp.unpackb(r.get('predmeans'), raw=True)
predvars = mnp.unpackb(r.get('predvars'), raw=True)
Xtest = mnp.unpackb(r.get('Xtest'), raw=True)
Xtraining = mnp.unpackb(r.get('Xtraining'), raw=True)

In [6]:
# Get latitude and longitude from Xtest and convert to mercator
lon = []
lat = []
for i in range(len(Xtest[0][1])):
    lon.extend(Xtest[0][i].tolist())
    lat.extend(Xtest[1][i].tolist())
x, y = merc(lon,lat)

In [7]:
lon, lat = lonlat(x,y)

In [8]:
# Create array of sizes for markers from uncertainty predictions
sizes = 15*np.log(1 + 300/np.sqrt(predvars))
min(sizes)

array([46.34758742])

In [9]:
filter = []
for var in predvars:
    if np.sqrt(var) > np.sqrt(predvars.mean()):
        filter.append(False)
    else:
        filter.append(True)

In [10]:
x_filtered = np.asarray(x)[filter]
y_filtered = np.asarray(y)[filter]
mean_filtered = predmeans[filter]
vars_filtered = predvars[filter]
sizes_filtered = 15*np.log(1 + 300/np.sqrt(vars_filtered))
#alphas = (0.2 + 0.8*min(vars_filtered)/vars_filtered)
alphas = (1/np.log(1 + np.sqrt(vars_filtered)))/max((1/np.log(1 + np.sqrt(vars_filtered))))
lon_filtered = np.asarray(lon)[filter]
lat_filtered = np.asarray(lat)[filter]

In [11]:
max(lat)


0.49999999999999434

In [12]:
x0, y0, xend, yend = [32.4], [0.01], [32.8], [0.5]
mercx0, mercy0 = merc(x0, y0)
mercxend, mercyend = merc(xend, yend)
gridsize = 20

In [31]:
source.data

AttributeError: 'tuple' object has no attribute 'data'

In [44]:
url = "http://127.0.0.1:5000/?xstart={}&xend={}&ystart={}&yend={}&gridsize={}".format(mercx0[0],
                                                                                      mercxend[0],
                                                                                      mercy0[0],
                                                                                      mercyend[0],
                                                                                      gridsize)
init_json = requests.get(url).json()

# Get map tiles
tile_provider = get_provider(CARTODBPOSITRON)

# Create ColumnDataSource from coordinates, sizes and predicted means
source = ColumnDataSource(dict(xgrid=init_json["xgrid"], 
                               ygrid=init_json["ygrid"], 
                               sizes=init_json["sizes"], 
                               means=init_json["means"], 
                               vars=np.sqrt(init_json["vars"]).tolist(), 
                               lat=init_json["lat"], 
                               lon=init_json["lon"]))

# Create linear colour map for pollution data
mapper = linear_cmap(field_name='means', palette=YlOrBr[9][::-1] ,low=0 ,high=max(init_json["means"]))

# Tooltips
tooltips = [("Lat/long", "@lat, @lon"),
            ("Predicted PM2.5", "@means"),
            ("Predicted Variance", "@vars")]

# Plot figure and add tiles 
scaled_map = figure(title=None, x_range=(min(x), max(x)), y_range=(min(y), max(y)),
           x_axis_type="mercator", y_axis_type="mercator", tooltips=tooltips, tools = "pan,wheel_zoom,box_zoom,reset,hover")

scaled_map.add_tile(tile_provider)


# Create and plot glyphs for predictions
glyph = Circle(x="xgrid", y="ygrid", size= 10, line_color="white", fill_color=mapper, fill_alpha=0.8, line_width=1, 
                line_alpha=1)
scaled_map.add_glyph(source, glyph)

# Add colour bar 
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
scaled_map.add_layout(color_bar, 'right')

callback = CustomJS(args=dict(source=source, xr=scaled_map.x_range, yr=scaled_map.y_range), code="""
    
    var xstart = xr.start;
    var xend = xr.end;
    var ystart = yr.start;
    var yend = yr.end;
    var gridsize = 20;
    var url = "http://127.0.0.1:5000/?xstart="+xstart+"&xend="+xend+"&ystart="+ystart+"&yend="+yend+"&gridsize="+gridsize;
    
    fetch(url)        
        .then(
            function(response) {
              response.json().then(function(data) {
                source.data = data;
                });
            }
      )
          
   
    """)

scaled_map.js_on_event(LODEnd, callback)

output_notebook()
output_file("scaled_map.html")
show(scaled_map)
# Save the plot by passing the plot -object and output path
#save(obj=scaled_map, filename=outfp)

Loading BokehJS ...

In [48]:
url = "https://us-central1-airqo-250220.cloudfunctions.net/get_interpolated/?xstart={}&xend={}&ystart={}&yend={}&gridsize={}".format(mercx0[0],
                                                                                      mercxend[0],
                                                                                      mercy0[0],
                                                                                      mercyend[0],
                                                                                      gridsize)
init_json = requests.get(url).json()

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [38]:
source["xgrid"]

TypeError: 'ColumnDataSource' object is not subscriptable

In [40]:
source.data["xgrid"]


[3606751.5017020637,
 3609095.0699292906,
 3611438.6381565174,
 3613782.2063837443,
 3616125.7746109706,
 3618469.3428381975,
 3620812.9110654243,
 3623156.479292651,
 3625500.047519878,
 3627843.615747105,
 3630187.1839743312,
 3632530.752201558,
 3634874.320428785,
 3637217.888656012,
 3639561.4568832386,
 3641905.0251104655,
 3644248.593337692,
 3646592.1615649187,
 3648935.7297921455,
 3651279.2980193724,
 3606751.5017020637,
 3609095.0699292906,
 3611438.6381565174,
 3613782.2063837443,
 3616125.7746109706,
 3618469.3428381975,
 3620812.9110654243,
 3623156.479292651,
 3625500.047519878,
 3627843.615747105,
 3630187.1839743312,
 3632530.752201558,
 3634874.320428785,
 3637217.888656012,
 3639561.4568832386,
 3641905.0251104655,
 3644248.593337692,
 3646592.1615649187,
 3648935.7297921455,
 3651279.2980193724,
 3606751.5017020637,
 3609095.0699292906,
 3611438.6381565174,
 3613782.2063837443,
 3616125.7746109706,
 3618469.3428381975,
 3620812.9110654243,
 3623156.479292651,
 362550